In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from convertbng.util import convert_bng, convert_lonlat

## Import Data 

You can download the datasets from [here](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data).

In [2]:
acc16 = pd.read_csv("accident-2016.csv", low_memory=False)
acc17 = pd.read_csv("accident-2017.csv", low_memory=False)
acc18 = pd.read_csv("accident-2018.csv", low_memory=False)
acc19 = pd.read_csv("accident-2019.csv", low_memory=False)
acc20 = pd.read_csv("accident-2020.csv", low_memory=False)

In [3]:
cas16 = pd.read_csv("casualty-2016.csv", low_memory=False)
cas17 = pd.read_csv("casualty-2017.csv", low_memory=False)
cas18 = pd.read_csv("casualty-2018.csv", low_memory=False)
cas19 = pd.read_csv("casualty-2019.csv", low_memory=False)
cas20 = pd.read_csv("casualty-2020.csv", low_memory=False)

In [4]:
veh16 = pd.read_csv("vehicle-2016.csv", low_memory=False)
veh17 = pd.read_csv("vehicle-2017.csv", low_memory=False)
veh18 = pd.read_csv("vehicle-2018.csv", low_memory=False)
veh19 = pd.read_csv("vehicle-2019.csv", low_memory=False)
veh20 = pd.read_csv("vehicle-2020.csv", low_memory=False)

The guide file is the `Road Safety Open Dataset Data Guide.xlsx` file available under the `Supporting Documents` section in the [link](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data).

In [5]:
guide = pd.read_excel("guide.xlsx")

In [6]:
guide.head()

,table,field name,code/format,label,note,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Accident,accident_index,NaN,NaN,unique value for each accident. The accident_i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Accident,accident_year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Accident,accident_reference,NaN,NaN,In year id used by the police to reference a c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Accident,location_easting_osgr,NaN,NaN,Null if not known,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Accident,location_northing_osgr,NaN,NaN,Null if not known,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Decode the data

Decode the encoded fields in the accident datasets - `acc16, acc17, acc18, acc19, acc20`

In [7]:
acc_encoded_fields = ['police_force','accident_severity','day_of_week',
                      'local_authority_district','local_authority_ons_district',
                      'local_authority_highway','first_road_class','road_type',
                     'junction_detail','junction_control','second_road_class',
                      'pedestrian_crossing_human_control','pedestrian_crossing_physical_facilities',
                     'light_conditions','weather_conditions','road_surface_conditions',
                     'special_conditions_at_site','carriageway_hazards',
                     'urban_or_rural_area','did_police_officer_attend_scene_of_accident',
                     'trunk_road_flag']

In [8]:
acc17[acc_encoded_fields].head()

,police_force,accident_severity,day_of_week,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,road_type,junction_detail,junction_control,...,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag
0,1,1,7,32,E09000010,E09000010,3,6,0,-1,...,0,0,4,1,1,0,0,1,1,2
1,1,3,1,1,E09000033,E09000033,3,6,3,4,...,0,0,4,1,2,0,0,1,1,2
2,1,3,1,5,E09000030,E09000030,3,6,3,4,...,0,0,4,1,1,0,0,1,1,2
3,1,3,1,32,E09000010,E09000010,3,1,1,4,...,0,4,4,2,2,0,0,1,1,2
4,1,2,1,4,E09000012,E09000012,3,3,6,2,...,0,5,4,1,2,0,0,1,1,2


In [9]:
acc_guide = guide.loc[ guide['table']=='Accident',:]
for ef in acc_encoded_fields:
    ef_guide = acc_guide.loc[ acc_guide['field name']==ef, :]
    codes = ef_guide['code/format'].values
    for code in codes:
        acc16.loc[acc16[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        acc17.loc[acc17[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        acc18.loc[acc18[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        acc19.loc[acc19[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        acc20.loc[acc20[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        

In [11]:
acc17[acc_encoded_fields].head()

,police_force,accident_severity,day_of_week,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,road_type,junction_detail,junction_control,...,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag
0,Metropolitan Police,Fatal,Saturday,Enfield,Enfield,Enfield,A,Single carriageway,Not at junction or within 20 metres,Data missing or out of range,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk
1,Metropolitan Police,Slight,Sunday,Westminster,Westminster,Westminster,A,Single carriageway,T or staggered junction,Give way or uncontrolled,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk
2,Metropolitan Police,Slight,Sunday,Tower Hamlets,Tower Hamlets,Tower Hamlets,A,Single carriageway,T or staggered junction,Give way or uncontrolled,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk
3,Metropolitan Police,Slight,Sunday,Enfield,Enfield,Enfield,A,Roundabout,Roundabout,Give way or uncontrolled,...,None within 50 metres,"Pelican, puffin, toucan or similar non-junctio...",Darkness - lights lit,Raining no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk
4,Metropolitan Police,Serious,Sunday,Hackney,Hackney,Hackney,A,Dual carriageway,Crossroads,Auto traffic signal,...,None within 50 metres,Pedestrian phase at traffic signal junction,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk


Decode the encoded fields in the casualty datasets - `cas16, cas17, cas18, cas19, cas20`

In [12]:
cas_encoded_fields = ['casualty_class','sex_of_casualty','age_band_of_casualty',
                     'casualty_severity','pedestrian_location','pedestrian_movement',
                     'car_passenger','bus_or_coach_passenger','pedestrian_road_maintenance_worker',
                     'casualty_type','casualty_imd_decile','casualty_home_area_type']

In [13]:
cas17[cas_encoded_fields].head()

,casualty_class,sex_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_imd_decile,casualty_home_area_type
0,2,2,4,3,0,0,1,0,0,9,2,1
1,1,1,4,2,0,0,0,0,0,2,-1,-1
2,2,1,4,1,0,0,0,0,0,2,-1,-1
3,2,2,6,3,0,0,1,0,0,9,5,1
4,1,2,6,3,0,0,0,0,0,9,5,1


In [14]:
cas_guide = guide.loc[ guide['table']=='Casualty',:]
for ef in cas_encoded_fields:
    ef_guide = cas_guide.loc[ cas_guide['field name']==ef, :]
    codes = ef_guide['code/format'].values
    for code in codes:
        cas16.loc[cas16[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        cas17.loc[cas17[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        cas18.loc[cas18[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        cas19.loc[cas19[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        cas20.loc[cas20[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        

In [15]:
cas17[cas_encoded_fields].head()

,casualty_class,sex_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_imd_decile,casualty_home_area_type
0,Passenger,Female,16 - 20,Slight,Not a Pedestrian,Not a Pedestrian,Front seat passenger,Not a bus or coach passenger,No / Not applicable,Car occupant,More deprived 10-20%,Urban area
1,Driver or rider,Male,16 - 20,Serious,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Motorcycle 50cc and under rider or passenger,Data missing or out of range,Data missing or out of range
2,Passenger,Male,16 - 20,Fatal,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Motorcycle 50cc and under rider or passenger,Data missing or out of range,Data missing or out of range
3,Passenger,Female,26 - 35,Slight,Not a Pedestrian,Not a Pedestrian,Front seat passenger,Not a bus or coach passenger,No / Not applicable,Car occupant,More deprived 40-50%,Urban area
4,Driver or rider,Female,26 - 35,Slight,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Car occupant,More deprived 40-50%,Urban area


Decode the encoded fields in the vehicles datasets - `veh16, veh17, veh18, veh19, veh20`

In [16]:
veh_encoded_fields = ['vehicle_type','towing_and_articulation','vehicle_manoeuvre',
                     'vehicle_direction_from','vehicle_direction_to','vehicle_location_restricted_lane',
                     'junction_location','skidding_and_overturning','hit_object_in_carriageway',
                     'vehicle_leaving_carriageway','hit_object_off_carriageway','first_point_of_impact',
                     'vehicle_left_hand_drive','journey_purpose_of_driver','sex_of_driver',
                     'age_band_of_driver','propulsion_code','driver_imd_decile',
                     'driver_home_area_type']

In [17]:
veh17[veh_encoded_fields].head()

,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_band_of_driver,propulsion_code,driver_imd_decile,driver_home_area_type
0,9,0,18,1,5,0,0,0,0,0,0,1,1,6,1,5,2,-1,-1
1,2,0,18,1,5,0,0,1,0,0,0,2,1,6,1,4,-1,-1,-1
2,9,0,18,5,1,0,1,0,0,0,0,2,1,6,1,6,8,9,1
3,9,0,18,5,1,0,1,1,0,0,0,1,1,6,1,7,2,2,1
4,9,0,18,3,7,0,1,0,0,0,0,1,1,6,3,-1,-1,-1,-1


In [18]:
veh_guide = guide.loc[ guide['table']=='Vehicle',:]
for ef in veh_encoded_fields:
    ef_guide = veh_guide.loc[ veh_guide['field name']==ef, :]
    codes = ef_guide['code/format'].values
    for code in codes:
        veh16.loc[veh16[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        veh17.loc[veh17[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        veh18.loc[veh18[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        veh19.loc[veh19[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        veh20.loc[veh20[ef] == code, ef] = ef_guide.loc[ guide['code/format']==code, 'label'].values[0]
        

In [19]:
veh17[veh_encoded_fields].head()

,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_band_of_driver,propulsion_code,driver_imd_decile,driver_home_area_type
0,Car,No tow/articulation,Going ahead other,North,South East,On main c'way - not in restricted lane,Not at or within 20 metres of junction,None,None,Did not leave carriageway,None,Front,No,Not known,Male,21 - 25,Heavy oil,Data missing or out of range,Data missing or out of range
1,Motorcycle 50cc and under,No tow/articulation,Going ahead other,North,South East,On main c'way - not in restricted lane,Not at or within 20 metres of junction,Skidded,None,Did not leave carriageway,None,Back,No,Not known,Male,16 - 20,Undefined,Data missing or out of range,Data missing or out of range
2,Car,No tow/articulation,Going ahead other,South East,North,On main c'way - not in restricted lane,Approaching junction or waiting/parked at junc...,None,None,Did not leave carriageway,None,Back,No,Not known,Male,26 - 35,Hybrid electric,Less deprived 10-20%,Urban area
3,Car,No tow/articulation,Going ahead other,South East,North,On main c'way - not in restricted lane,Approaching junction or waiting/parked at junc...,Skidded,None,Did not leave carriageway,None,Front,No,Not known,Male,36 - 45,Heavy oil,More deprived 10-20%,Urban area
4,Car,No tow/articulation,Going ahead other,East,West,On main c'way - not in restricted lane,Approaching junction or waiting/parked at junc...,None,None,Did not leave carriageway,None,Front,No,Not known,Not known,Data missing or out of range,Undefined,Data missing or out of range,Data missing or out of range


## Save decoded data

In [20]:
acc16.to_csv("decoded/accident-2016.csv", index=False)
acc17.to_csv("decoded/accident-2017.csv", index=False)
acc18.to_csv("decoded/accident-2018.csv", index=False)
acc19.to_csv("decoded/accident-2019.csv", index=False)
acc20.to_csv("decoded/accident-2020.csv", index=False)

cas16.to_csv("decoded/casualty-2016.csv", index=False)
cas17.to_csv("decoded/casualty-2017.csv", index=False)
cas18.to_csv("decoded/casualty-2018.csv", index=False)
cas19.to_csv("decoded/casualty-2019.csv", index=False)
cas20.to_csv("decoded/casualty-2020.csv", index=False)

veh16.to_csv("decoded/vehicle-2016.csv", index=False)
veh17.to_csv("decoded/vehicle-2017.csv", index=False)
veh18.to_csv("decoded/vehicle-2018.csv", index=False)
veh19.to_csv("decoded/vehicle-2019.csv", index=False)
veh20.to_csv("decoded/vehicle-2020.csv", index=False)

## EDA

In [29]:
print("\n********ACCIDENT DATASET********")
print(acc16.shape)
print(acc17.shape)
print(acc18.shape)
print(acc19.shape)
print(acc20.shape)

print("\n********CASUALTY DATASET********")
print(cas16.shape)
print(cas17.shape)
print(cas18.shape)
print(cas19.shape)
print(cas20.shape)

print("\n********VEHICLE DATASET********")
print(veh16.shape)
print(veh17.shape)
print(veh18.shape)
print(veh19.shape)
print(veh20.shape)

print("\n********GUIDE DATASET********")
print(guide.shape)


********ACCIDENT DATASET********
(136621, 36)
(129982, 36)
(122635, 36)
(117536, 36)
(91199, 36)

********CASUALTY DATASET********
(181384, 18)
(170993, 18)
(160597, 18)
(153158, 18)
(115584, 18)

********VEHICLE DATASET********
(252500, 27)
(238926, 27)
(226409, 27)
(216381, 27)
(167375, 27)

********GUIDE DATASET********
(1579, 5)


## Import decoded data

In [2]:
acc16 = pd.read_csv("decoded/accident-2016.csv", low_memory=False)
acc17 = pd.read_csv("decoded/accident-2017.csv", low_memory=False)
acc18 = pd.read_csv("decoded/accident-2018.csv", low_memory=False)
acc19 = pd.read_csv("decoded/accident-2019.csv", low_memory=False)
acc20 = pd.read_csv("decoded/accident-2020.csv", low_memory=False)

In [3]:
len(acc16.columns)

36

In [4]:
acc16.head()

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2.02E+12,2016,10000005,519310.0,188730.0,-0.279323,51.584754,Metropolitan Police,Slight,2,...,No physical crossing facilities within 50 metres,Darkness - lights unlit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01000543
1,2.02E+12,2016,10000006,551920.0,174560.0,0.184928,51.449595,Metropolitan Police,Slight,1,...,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01000375
2,2.02E+12,2016,10000008,505930.0,183850.0,-0.473837,51.543563,Metropolitan Police,Slight,1,...,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01033725
3,2.02E+12,2016,10000016,527770.0,168930.0,-0.164442,51.404958,Metropolitan Police,Slight,1,...,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01003379
4,2.02E+12,2016,10000018,510740.0,177230.0,-0.406580,51.483139,Metropolitan Police,Slight,2,...,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01002583


## Add location data to accidents dataset based on Lat and Long - Dont run
#### This approach was later dropped since it was unable to handle the large amount of API calls that need to be made

In [48]:
# acc16['road'] = 'No data'
# acc16['city'] = 'No data'
# acc16['county'] = 'No data'
# acc16['postcode'] = 'No data'
# acc16['country'] = 'No data'

# acc17['road'] = 'No data'
# acc17['city'] = 'No data'
# acc17['county'] = 'No data'
# acc17['postcode'] = 'No data'
# acc17['country'] = 'No data'

# acc18['road'] = 'No data'
# acc18['city'] = 'No data'
# acc18['county'] = 'No data'
# acc18['postcode'] = 'No data'
# acc18['country'] = 'No data'

# acc19['road'] = 'No data'
# acc19['city'] = 'No data'
# acc19['county'] = 'No data'
# acc19['postcode'] = 'No data'
# acc19['country'] = 'No data'

In [44]:
# geolocator = Nominatim(user_agent="geoapiExercises")

In [53]:
# def getLocationDetails(row):
#     if (str(row['latitude']) != 'nan') and (str(row['longitude']) != 'nan'):
# #         print(format(float(row['latitude']), '.8f')+","+format(float(row['longitude']), '.8f'))
#         location = geolocator.reverse(format(float(row['latitude']), '.8f')+","+format(float(row['longitude']), '.8f')).raw["address"]
#         try:
#             if 'road' in location:
#                 row['road'] = location['road']
#             if 'city' in location:
#                 row['city'] = location['city']
#             if 'county' in location:
#                 row['county'] = location['county']
#             if 'postcode' in location:
#                 row['postcode'] = location['postcode']
#             if 'country' in location:
#                 row['country'] = location['country']
#         except KeyError:
#             print(str(row['accident_index'])+","+str(row['latitude'])+","+str(row['longitude']))
#         except ValueError:
#             print(str(row['accident_index'])+","+str(row['latitude'])+","+str(row['longitude']))

In [5]:
# acc16.apply(lambda row: getLocationDetails(row), axis=1)

In [ ]:
# # convert lists of BNG Eastings and Northings to longitude, latitude
# eastings = [539250.0]
# northings = [167780.0]
# res_list_en = convert_lonlat(eastings, northings)
# print(res_list_en)

## Group districts to Major cities

In [4]:
acc16['city'] = 'No data'
acc16['county'] = 'No data'
acc16['country'] = 'No data'

acc17['city'] = 'No data'
acc17['county'] = 'No data'
acc17['country'] = 'No data'

acc18['city'] = 'No data'
acc18['county'] = 'No data'
acc18['country'] = 'No data'

acc19['city'] = 'No data'
acc19['county'] = 'No data'
acc19['country'] = 'No data'

acc20['city'] = 'No data'
acc20['county'] = 'No data'
acc20['country'] = 'No data'

geolocator = Nominatim(user_agent="geoapiExercises")

In [6]:
local_authority_district = set()

In [7]:
local_authority_district.update(acc16['local_authority_district'].unique())
local_authority_district.update(acc17['local_authority_district'].unique())
local_authority_district.update(acc18['local_authority_district'].unique())
local_authority_district.update(acc19['local_authority_district'].unique())
local_authority_district.update(acc20['local_authority_district'].unique())
local_authority_district.remove('-1')

In [8]:
len(local_authority_district)

380

In [ ]:
# district_city_mapping = {}
# def getCityForDistrict(row):
#     if (str(row['latitude']) != 'nan') and (str(row['longitude']) != 'nan'):
# #         print(format(float(row['latitude']), '.8f')+","+format(float(row['longitude']), '.8f'))
#         location = geolocator.reverse(format(float(row['latitude']), '.8f')+","+format(float(row['longitude']), '.8f')).raw["address"]
#         try:
#             if 'road' in location:
#                 row['road'] = location['road']
#             if 'city' in location:
#                 row['city'] = location['city']
#             if 'county' in location:
#                 row['county'] = location['county']
#             if 'postcode' in location:
#                 row['postcode'] = location['postcode']
#             if 'country' in location:
#                 row['country'] = location['country']
#         except KeyError:
#             print(str(row['accident_index'])+","+str(row['latitude'])+","+str(row['longitude']))
#         except ValueError:
#             print(str(row['accident_index'])+","+str(row['latitude'])+","+str(row['longitude']))

In [ ]:
i = 0
for district in local_authority_district:
    print("Fetching details of: {}".format(district))
    location = {}
    index = 1
    district_df = acc16.loc[acc16['local_authority_district'] == district,['longitude', 'latitude']]
    while (('county' not in location) or ('city' not in location) or ('country' not in location)) and index<10:
#         print(index, location)
        district_df_1 = district_df.iloc[index,:]
        location = geolocator.reverse(format(float(district_df_1['latitude']), '.8f')+","+format(float(district_df_1['longitude']), '.8f')).raw["address"]
        index += 1
    if 'county' in location:
        acc16.loc[acc16['local_authority_district'] == district, 'county'] = location['county']
        acc17.loc[acc17['local_authority_district'] == district, 'county'] = location['county']
        acc18.loc[acc18['local_authority_district'] == district, 'county'] = location['county']
        acc19.loc[acc19['local_authority_district'] == district, 'county'] = location['county']
        acc20.loc[acc20['local_authority_district'] == district, 'county'] = location['county']
    
    if 'city' in location:
        acc16.loc[acc16['local_authority_district'] == district, 'city'] = location['city']
        acc17.loc[acc17['local_authority_district'] == district, 'city'] = location['city']
        acc18.loc[acc18['local_authority_district'] == district, 'city'] = location['city']
        acc19.loc[acc19['local_authority_district'] == district, 'city'] = location['city']
        acc20.loc[acc20['local_authority_district'] == district, 'city'] = location['city']
    
    if 'country' in location:
        acc16.loc[acc16['local_authority_district'] == district, 'country'] = location['country']
        acc17.loc[acc17['local_authority_district'] == district, 'country'] = location['country']
        acc18.loc[acc18['local_authority_district'] == district, 'country'] = location['country']
        acc19.loc[acc19['local_authority_district'] == district, 'country'] = location['country']
        acc20.loc[acc20['local_authority_district'] == district, 'country'] = location['country']
    i += 1
    if i//10:
        print(i)
print(i)

# Cornwall is among the top districts. County is the same. So we hardcode the city as well as Cornwall
acc16.loc[acc16['local_authority_district']=='Cornwall',['city']] = 'Cornwall'
acc17.loc[acc17['local_authority_district']=='Cornwall',['city']] = 'Cornwall'
acc18.loc[acc18['local_authority_district']=='Cornwall',['city']] = 'Cornwall'
acc19.loc[acc19['local_authority_district']=='Cornwall',['city']] = 'Cornwall'
acc20.loc[acc20['local_authority_district']=='Cornwall',['city']] = 'Cornwall'

# Replace 'City of London' and 'London Borough of Lambeth' with 'London' in city
acc16.loc[acc16['city']=='City of London','city'] = 'London'
acc17.loc[acc17['city']=='City of London','city'] = 'London'
acc18.loc[acc18['city']=='City of London','city'] = 'London'
acc19.loc[acc19['city']=='City of London','city'] = 'London'
acc20.loc[acc20['city']=='City of London','city'] = 'London'

acc16.loc[acc16['city']=='London Borough of Lambeth','city'] = 'London'
acc17.loc[acc17['city']=='London Borough of Lambeth','city'] = 'London'
acc18.loc[acc18['city']=='London Borough of Lambeth','city'] = 'London'
acc19.loc[acc19['city']=='London Borough of Lambeth','city'] = 'London'
acc20.loc[acc20['city']=='London Borough of Lambeth','city'] = 'London'

In [49]:
acc16.to_csv("decoded/accident-2016.csv", index=False)
acc17.to_csv("decoded/accident-2017.csv", index=False)
acc18.to_csv("decoded/accident-2018.csv", index=False)
acc19.to_csv("decoded/accident-2019.csv", index=False)
acc20.to_csv("decoded/accident-2020.csv", index=False)